In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

56 documents
17 classes ['appreciate', 'call nt picking', 'email', 'funny', 'goodbye', 'greeting', 'joke', 'made', 'not_accepting', 'offers', 'order not recieved', 'payment failure', 'payments', 'phone', 'thanks', 'wrong items', 'yourself']
105 unique lemmatized words ["'d", "'s", 'a', 'about', 'accept', 'after', 'agent', 'and', 'anyone', 'are', 'avail', 'back', 'bye', 'call', 'can', 'cash', 'connect', 'contact', 'created', 'credit', 'day', 'detail', 'did', 'didnt', 'different', 'do', 'email', 'executive', 'failed', 'failure', 'for', 'funny', 'get', 'give', 'good', 'goodbye', 'got', 'ha', 'have', 'he', 'hear', 'hello', 'helpful', 'hi', 'how', 'i', 'is', 'it', 'item', 'joke', 'later', 'like', 'made', 'me', 'method', 'missing', 'money', 'my', 'name', 'next', 'nice', 'not', 'now', 'number', 'offer', 'ok', 'online', 'only', 'order', 'ordered', 'payment', 'phone', 'picking', 'please', 'pls', 'received', 'recieved', 'refund', 'same', 'say', 'see', 'some', 'step', 'take', 'tell', 'thank', 'th

In [6]:
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\saura\AppData\Local\Temp\ipykernel_19312\3528693901.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [10]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

Epoch 1/200
12/12 [==============================] - 5s 16ms/step - loss: 2.8691 - accuracy: 0.0893
Epoch 2/200
12/12 [==============================] - 0s 6ms/step - loss: 2.8756 - accuracy: 0.0536
Epoch 3/200
12/12 [==============================] - 0s 6ms/step - loss: 2.7814 - accuracy: 0.1071
Epoch 4/200
12/12 [==============================] - 0s 6ms/step - loss: 2.7082 - accuracy: 0.1429
Epoch 5/200
12/12 [==============================] - 0s 6ms/step - loss: 2.6373 - accuracy: 0.1964
Epoch 6/200
12/12 [==============================] - 0s 5ms/step - loss: 2.5610 - accuracy: 0.3036
Epoch 7/200
12/12 [==============================] - 0s 6ms/step - loss: 2.5478 - accuracy: 0.1786
Epoch 8/200
12/12 [==============================] - 0s 5ms/step - loss: 2.3906 - accuracy: 0.3393
Epoch 9/200
12/12 [==============================] - 0s 6ms/step - loss: 2.3030 - accuracy: 0.3750
Epoch 10/200
12/12 [==============================] - 0s 3ms/step - loss: 2.1957 - accuracy: 0.4464
Epoch 11

12/12 [==============================] - 0s 5ms/step - loss: 0.1186 - accuracy: 0.9643
Epoch 84/200
12/12 [==============================] - 0s 4ms/step - loss: 0.2398 - accuracy: 0.9107
Epoch 85/200
12/12 [==============================] - 0s 4ms/step - loss: 0.1868 - accuracy: 0.9643
Epoch 86/200
12/12 [==============================] - 0s 4ms/step - loss: 0.2888 - accuracy: 0.8929
Epoch 87/200
12/12 [==============================] - 0s 4ms/step - loss: 0.3583 - accuracy: 0.9286
Epoch 88/200
12/12 [==============================] - 0s 4ms/step - loss: 0.1404 - accuracy: 0.9464
Epoch 89/200
12/12 [==============================] - 0s 5ms/step - loss: 0.4069 - accuracy: 0.9107
Epoch 90/200
12/12 [==============================] - 0s 4ms/step - loss: 0.2264 - accuracy: 0.9464
Epoch 91/200
12/12 [==============================] - 0s 4ms/step - loss: 0.1480 - accuracy: 0.9643
Epoch 92/200
12/12 [==============================] - 0s 4ms/step - loss: 0.0841 - accuracy: 0.9821
Epoch 93/200


12/12 [==============================] - 0s 4ms/step - loss: 0.0544 - accuracy: 0.9821
Epoch 165/200
12/12 [==============================] - 0s 4ms/step - loss: 0.0642 - accuracy: 1.0000
Epoch 166/200
12/12 [==============================] - 0s 4ms/step - loss: 0.1649 - accuracy: 0.9464
Epoch 167/200
12/12 [==============================] - 0s 5ms/step - loss: 0.0267 - accuracy: 1.0000
Epoch 168/200
12/12 [==============================] - 0s 4ms/step - loss: 0.0452 - accuracy: 0.9821
Epoch 169/200
12/12 [==============================] - 0s 5ms/step - loss: 0.0801 - accuracy: 0.9821
Epoch 170/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0768 - accuracy: 1.0000
Epoch 171/200
12/12 [==============================] - 0s 4ms/step - loss: 0.0238 - accuracy: 1.0000
Epoch 172/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0506 - accuracy: 1.0000
Epoch 173/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0140 - accuracy: 1.0000
Epoc

In [11]:
print("Model created")

Model created
